In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

# Loading the other modules I have in parent directories
import sys
PYTHONPATH="../"
sys.path.append(PYTHONPATH)
from dataProcessing import getDataLoaders
from models import ThreeCNN
from train import check_accuracy, train, train_ThreeCNN
from plottingFcts import trainingMetrics, sigBkgEff, plotROC

# Some more magic so that the notebook will reload external python modules;
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

/home/shared/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [5]:
loader_train, loader_val, loader_test = getDataLoaders(batch_size=256)

OSError: Unable to open file (unable to open file: name = '../data/gamma.hdf5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

Let's try to set up a basic model and train it for one epoch, to make sure the code works.

In [9]:
# establish network dimensions 
layer0_params = [1, 3, 3, 1, 1]
layer0_reduce = [1, 3, 93, 1, 1]

layer1_params = [1, 3, 3, 1, 1]
layer1_reduce = [1, 12, 12, 1, 1]

layer2_params = [1, 3, 3, 1, 1]
layer2_reduce = [1, 12, 3, 1, 1]

fc_params = [54, 150, 100, 50, 3, 0.5]

test_one = ThreeCNN(layer0_params, layer0_reduce, layer1_params, layer1_reduce, layer2_params, layer2_reduce, fc_params)

In [12]:
# a check that the model parameters are correct 
print(test_one.layer0_params_all)
# for some reason, duplicate values (1's) are not being appended--why? 

[1, 3, 3, 1, 1, 3, 93, 1]


In [13]:
# construct layer models, FC model 
layer_models = test_one.forward_preprocess() 
fc_model = test_one.forward_fc()

IndexError: list index out of range

In [ ]:
# unpack layer models 
layer0_model = layer_models[0]
layer1_model = layer_models[1]
layer2_model = layer_models[2] 

In [ ]:
# optimizer 
optimizer = optim.Adam(
    [{'params' : layer0_model.parameters()},
    {'params' : layer1_model.parameters()},
    {'params' : layer2_model.parameters()},
    {'params' : fc_model.parameters()}]
)
# train the network for one epoch 
train_ThreeCNN(loader_train, loader_val, layer0_model, layer1_model, layer2_model, fc_model, optimizer)